In [12]:
import sys
import stablemotifs
import biolqm
import ginsim
import pystablemotifs as sm
import pyboolnet
import boolsim
import timeit
import timeout_decorator
from timeout_decorator import TimeoutError
import pandas as pd
import glob
from colomoto_jupyter import tabulate
import numpy as np


## Generating Random Boolean Networks for the benchmarks

In [2]:

rbn_models_path='rbn_models_for_benchmark/'
generate_new_models=True
#this part populates the folder given in rbn_models_path with random boolean networks
#WARNING! the folder already has models in it from previous runs, those will be ran too.
#for a completely fresh run, empty the folder, or specify the model files considered explicitly below in the model_set
if generate_new_models:
    K=2
    p_bias=0.5
    N_ensemble=10 # the number of networks of the same size
    seed=1991
    for N in range(10,60,10): #the range of sizes
        rbn_ensemble_rules=sm.random_boolean_networks.random_boolean_network_ensemble_kauffman(N,K,p_bias,N_ensemble,seed=seed,write_boolean_network=False)
        for i in range(N_ensemble):
            with open(rbn_models_path+'rbn_N_%d_ens_%d.booleannet'%(N,i),'w') as f:
                f.write(rbn_ensemble_rules[i])
        

In [3]:

#run_ens_size is the number of times each method is repeatedly run IN THE SAME ITERATION. 
#this means that the timeout has to be increased proporionally to account for all runs
#In the final output the minimum (best) runtime is saved out of all in the ensemble 
run_ens_size=1 

#timeout limit in seconds for each method (all runs within the run_ens_size range). 
#If the timeout limit is reached the output will be NaN
timeout_limit=60 #seconds


#name of methods to be exluded from the run. This can be set manually if we know 
#that some methods are not relevant or take too much time
exclude_from_run=set(['bioLQM',
                  #    'pyboolnet',
                      'Pint',
                      #'stablemotifs2013',
                      #'pystablemotifs',
                      #'boolsim',
                      #'CABEAN'
                    ])
#exclude_from_run=set(['bioLQM','PyBoolNet_Asp','PyBoolNet','Pint']) 
#uncomment the line abovr if you want only methods that look for complex attractors

#the excel file in which the dataframe summarizing the results will be saved.
#if it already exists new results will be added to it
output_file_name='timing_results_RBN.xlsx'

In [4]:

#grabbing all the .booleannet models from the folder
model_set=glob.glob(rbn_models_path+'*.booleannet')
#model_set=['PSO.txt', 'phase_switch.txt','myeloid.txt']

for model_file_name in model_set:
    print('Running model %s'%model_file_name)
model_file_name='%s'%model_file_name
print(model_file_name)
with open(model_file_name) as f:
    rules=f.read().replace('#BOOLEAN RULES','').strip()
biolqm_model = biolqm.load(model_file_name, "booleannet")
pypint_model=biolqm.to_pint(biolqm_model)
print(biolqm_model)


Running model rbn_models_for_benchmark/rbn_N_50_ens_1.booleannet
Running model rbn_models_for_benchmark/rbn_N_40_ens_8.booleannet
Running model rbn_models_for_benchmark/rbn_N_10_ens_6.booleannet
Running model rbn_models_for_benchmark/rbn_N_10_ens_3.booleannet
Running model rbn_models_for_benchmark/rbn_N_20_ens_7.booleannet
Running model rbn_models_for_benchmark/rbn_N_30_ens_2.booleannet
Running model rbn_models_for_benchmark/rbn_N_20_ens_5.booleannet
Running model rbn_models_for_benchmark/rbn_N_40_ens_4.booleannet
Running model rbn_models_for_benchmark/rbn_N_30_ens_5.booleannet
Running model rbn_models_for_benchmark/rbn_N_40_ens_2.booleannet
Running model rbn_models_for_benchmark/rbn_N_30_ens_9.booleannet
Running model rbn_models_for_benchmark/rbn_N_40_ens_6.booleannet
Running model rbn_models_for_benchmark/rbn_N_20_ens_8.booleannet
Running model rbn_models_for_benchmark/rbn_N_10_ens_1.booleannet
Running model rbn_models_for_benchmark/rbn_N_30_ens_8.booleannet
Running model rbn_models_

org.colomoto.biolqm.LogicalModelImpl@784ac12


In [5]:
https://www.youtube.com/watch?v=NYhKmvglrxk&t=4s@timeout_decorator.timeout(timeout_limit)
def run_timed_test(method_runstring, setup, repeat):

    return timeit.repeat(method_runstring,setup=setup, repeat=repeat, number=1)

try:
    result_df=pd.read_excel(output_file_name)
    result_df=result_df.reset_index(drop=True)
    already_ran_=result_df.model
except FileNotFoundError:
    result_df=pd.DataFrame()
    already_ran_=[]


In [6]:
#grabbing all the .booleannet models from the folder
model_set=glob.glob(rbn_models_path+'*.booleannet')
#model_set=['PSO.txt', 'phase_switch.txt','myeloid.txt']


In [7]:
list(already_ran_)

[]

In [8]:
for model_file_name in sorted(model_set):
    if model_file_name.split('/')[-1].split('.')[0] in list(already_ran_):
        print(model_file_name + ' already in models ran, moving on to the next model')
        continue
    
    print('Running model %s'%model_file_name)
    
    base_setup_string='''import stablemotifs
import biolqm
import json
import ginsim
import pystablemotifs as sm
import pyboolnet
import boolsim
import cabean__version__
import pypint
from colomoto_jupyter import tabulate


model_file_name='%s'
with open(model_file_name) as f:
    rules=f.read().replace('#BOOLEAN RULES','').strip()
biolqm_model = biolqm.load(model_file_name, "booleannet")
pypint_model=biolqm.to_pint(biolqm_model)'''%model_file_name

    method_runstrings={'stablemotifs2013': "a=stablemotifs.load(biolqm_model, quiet=True).attractors\ntabulate(a).to_excel('attr_output/attr_output_stablemotifs2013_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'pystablemotifs': "ar = sm.AttractorRepertoire.from_primes(primes, max_simulate_size=max_simulate_size)\nex.attractor_dataframe(ar).to_excel('attr_output/attr_output_pystablemotifs_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')\nprint(ar.fewest_attractors,ar.most_attractors)",
                   
                   
 'pyboolnet': 'd=compute_attractors(primes, "asynchronous")\nwith open(\'attr_output/attr_output_pyboolnet_\'+model_file_name.split(\'.\')[0].split(\'/\')[-1]+\'.json\',\'w\') as fp:\n\tjson.dump(d,fp)',
                   
                   
 'boolsim': "a=boolsim.attractors(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_boolsim_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'bioLQM': "a=biolqm.fixpoints(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_bioLQM_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                                      
                   
 'CABEAN': "a=cabean.attractors(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_CABEAN_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'Pint': "a=pypint.fixpoints(pypint_model)\ntabulate(a).to_excel('attr_output/attr_output_Pint_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')", 
                      
  }

    method_setups={'stablemotifs2013':base_setup_string,

    'pystablemotifs':base_setup_string+'''
max_simulate_size=20
rules_pbn = sm.format.booleannet2bnet(rules)
primes = pyboolnet.prime_implicants.bnet_text2primes(rules_pbn)
import pystablemotifs.export as ex
''',

    'pyboolnet':base_setup_string+'''
rules_pbn = sm.format.booleannet2bnet(rules)
primes = pyboolnet.prime_implicants.bnet_text2primes(rules_pbn)
from pyboolnet.attractors import compute_attractors
''',

    'boolsim':base_setup_string,

    'bioLQM':base_setup_string,

    'CABEAN':base_setup_string,

    'Pint':base_setup_string}

    method_runtimes={i:[] for i in method_runstrings}
    
    
    for method_name in set(method_runstrings)-exclude_from_run:
        print('method:',method_name)
        try:
            method_runtimes[method_name]=run_timed_test(method_runstrings[method_name], setup=method_setups[method_name], repeat=run_ens_size)
        except Exception as e:
            print(str(e))
            method_runtimes[method_name].append(np.nan)
    runtime_minimums={k:min(method_runtimes[k]) for k in method_runtimes.keys()-exclude_from_run} 
    runtime_minimums['model']=model_file_name.split('.')[0].split('/')[-1]
    result_df=result_df.append([runtime_minimums],ignore_index=True)
    result_df=result_df.reset_index(drop=True)

    result_df.to_excel(output_file_name)

Running model rbn_models_for_benchmark/rbn_N_10_ens_0.booleannet
method: stablemotifs2013
method: pystablemotifs
1 1
method: boolsim
method: pyboolnet
INFO attractors.completeness(..)
INFO created /tmp/pyboolnet_pc5nlzu_
INFO yes
INFO  working on minimal trapspace 1/1: 0011011001
INFO attractors.univocality(..)
INFO  yes
INFO attractors.faithfulness(..)
INFO yes
INFO attractors.find_attractor_state_by_randomwalk_and_ctl(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=10, update=asynchronous, length=100, attempts=10
INFO trial 0
INFO start: 0011011001
INFO end: 0011011001
INFO created /tmp/pyboolnet_gxb75r9b
INFO is attractor state
method: CABEAN
Running model rbn_models_for_benchmark/rbn_N_10_ens_1.booleannet
method: stablemotifs2013
method: pystablemotifs
1 1
method: boolsim
method: pyboolnet
INFO attractors.completeness(..)
INFO created /tmp/pyboolnet_lx70i2yo
INFO yes
INFO  working on minimal trapspace 1/1: 1100011010
INFO attractors.univocality(..)
INFO  ye

INFO len(primes)=2, update=asynchronous, length=20, attempts=10
INFO trial 0
INFO start: 11
INFO end: 11
INFO created /tmp/pyboolnet_clljnej1
INFO is attractor state
INFO created /tmp/pyboolnet_lq09knu8
INFO  yes
INFO attractors.faithfulness(..)
INFO created /tmp/pyboolnet_ui_p2q5c
INFO yes
INFO attractors.find_attractor_state_by_randomwalk_and_ctl(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=10, update=asynchronous, length=100, attempts=10
INFO trial 0
INFO start: 0100010101
INFO end: 0110000101
INFO created /tmp/pyboolnet_yuy7iz7f
INFO is attractor state
INFO  working on minimal trapspace 2/2: ---11--01-
INFO attractors.univocality(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=6, update=asynchronous, length=60, attempts=10
INFO trial 0
INFO start: 100011
INFO end: 001101
INFO created /tmp/pyboolnet_unoa82ks
INFO is attractor state
INFO created /tmp/pyboolnet_8wuzf1ls
INFO  yes
INFO attractors.faithfulness(..)
INFO created /tmp/py

INFO len(primes)=20, update=asynchronous, length=200, attempts=10
INFO trial 0
INFO start: 01010011010101011101
INFO end: 01010011010101011101
INFO created /tmp/pyboolnet_7t2a5fdq
INFO is attractor state
method: CABEAN
Running model rbn_models_for_benchmark/rbn_N_20_ens_2.booleannet
method: stablemotifs2013
method: pystablemotifs
1 1
method: boolsim
method: pyboolnet
INFO attractors.completeness(..)
INFO created /tmp/pyboolnet_upr3gcfl
INFO created /tmp/pyboolnet_hk8qr9b1
INFO yes
INFO  working on minimal trapspace 1/1: 10001011001001-1-011
INFO attractors.univocality(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=2, update=asynchronous, length=20, attempts=10
INFO trial 0
INFO start: 10
INFO end: 01
INFO created /tmp/pyboolnet_hohdi71r
INFO is attractor state
INFO created /tmp/pyboolnet_7p0d1p9q
INFO  yes
INFO attractors.faithfulness(..)
INFO created /tmp/pyboolnet_6itaqe_s
INFO yes
INFO attractors.find_attractor_state_by_randomwalk_and_ctl(..)
INFO find_attr

INFO  working on minimal trapspace 2/4: 10010101001100011011
INFO attractors.univocality(..)
INFO  yes
INFO attractors.faithfulness(..)
INFO yes
INFO attractors.find_attractor_state_by_randomwalk_and_ctl(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=20, update=asynchronous, length=200, attempts=10
INFO trial 0
INFO start: 10010101001100011011
INFO end: 10010101001100011011
INFO created /tmp/pyboolnet_wjxy6p4d
INFO is attractor state
INFO  working on minimal trapspace 3/4: 00010101001100010111
INFO attractors.univocality(..)
INFO  yes
INFO attractors.faithfulness(..)
INFO yes
INFO attractors.find_attractor_state_by_randomwalk_and_ctl(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=20, update=asynchronous, length=200, attempts=10
INFO trial 0
INFO start: 00010101001100010111
INFO end: 00010101001100010111
INFO created /tmp/pyboolnet__ubqua_b
INFO is attractor state
INFO  working on minimal trapspace 4/4: 00010101101101010111
INFO attrac

INFO trial 0
INFO start: 001011000001010111111001011100
INFO end: 001011000001010111111001011100
INFO created /tmp/pyboolnet_q_6p5fi9
INFO is attractor state
INFO  working on minimal trapspace 2/2: 101011011110001010101011010010
INFO attractors.univocality(..)
INFO  yes
INFO attractors.faithfulness(..)
INFO yes
INFO attractors.find_attractor_state_by_randomwalk_and_ctl(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=30, update=asynchronous, length=300, attempts=10
INFO trial 0
INFO start: 101011011110001010101011010010
INFO end: 101011011110001010101011010010
INFO created /tmp/pyboolnet_qoen3_4q
INFO is attractor state
method: CABEAN
Running model rbn_models_for_benchmark/rbn_N_30_ens_6.booleannet
method: stablemotifs2013
method: pystablemotifs
1 1
method: boolsim
method: pyboolnet
INFO attractors.completeness(..)
INFO yes
INFO  working on minimal trapspace 1/1: ----0010100-01-0-0110-1101-110
INFO attractors.univocality(..)
INFO find_attractor_state_by_randomwa

INFO len(primes)=40, update=asynchronous, length=400, attempts=10
INFO trial 0
INFO start: 1000001010101110111111101110101001101101
INFO end: 1000001010101110111111101110101001101101
INFO created /tmp/pyboolnet_fid8mlt7
'Timed Out'
method: CABEAN
Running model rbn_models_for_benchmark/rbn_N_40_ens_1.booleannet
method: stablemotifs2013
method: pystablemotifs
1 1
method: boolsim
method: pyboolnet
INFO attractors.completeness(..)
INFO created /tmp/pyboolnet_g7wlfbm0
INFO created /tmp/pyboolnet_5ug314_s
INFO created /tmp/pyboolnet_tr0i8v2o
INFO created /tmp/pyboolnet_qm_akvl5
INFO yes
INFO  working on minimal trapspace 1/1: 100010110--100-01010110110---0-010111100
INFO attractors.univocality(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=7, update=asynchronous, length=70, attempts=10
INFO trial 0
INFO start: 1010001
INFO end: 1000000
INFO created /tmp/pyboolnet_ckxds73v
INFO is attractor state
INFO created /tmp/pyboolnet__yven3ub
INFO  yes
INFO attractors.faithful

INFO yes
INFO attractors.find_attractor_state_by_randomwalk_and_ctl(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=40, update=asynchronous, length=400, attempts=10
INFO trial 0
INFO start: 1100010000100100011010010111000111101001
INFO end: 1100010000100100011010010111000111101001
INFO created /tmp/pyboolnet_wygiwu6k
INFO is attractor state
method: CABEAN
Running model rbn_models_for_benchmark/rbn_N_40_ens_6.booleannet
method: stablemotifs2013
'Timed Out'
method: pystablemotifs
1 1
method: boolsim
method: pyboolnet
INFO attractors.completeness(..)
INFO created /tmp/pyboolnet_z9hjokk8
INFO yes
INFO  working on minimal trapspace 1/1: 1111000000000100110110110010100011111011
INFO attractors.univocality(..)
INFO  yes
INFO attractors.faithfulness(..)
INFO yes
INFO attractors.find_attractor_state_by_randomwalk_and_ctl(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=40, update=asynchronous, length=400, attempts=10
INFO trial 0
INFO start: 1111

INFO trial 0
INFO start: 1000110101110101011111100
INFO end: 1011000000010111110001010
INFO created /tmp/pyboolnet_1s_ouon_
INFO is attractor state
INFO created /tmp/pyboolnet_fxbzi144
INFO  yes
INFO attractors.faithfulness(..)
INFO created /tmp/pyboolnet_1hoi4dq7
INFO yes
INFO attractors.find_attractor_state_by_randomwalk_and_ctl(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=50, update=asynchronous, length=500, attempts=10
INFO trial 0
INFO start: 00000110000101101011001010101101110000100110110011
INFO end: 00111011000101101011000000101101111110100101110110
INFO created /tmp/pyboolnet_bly56hsg
'Timed Out'
method: CABEAN
Command 'cabean -asynbn -steadystates -newtarjan -newpred -compositional 2 /tmp/cabeankjqamm9g.ispl' returned non-zero exit status -11
Command line: cabean -asynbn -steadystates -newtarjan -newpred -compositional 2 /tmp/cabeankjqamm9g.ispl

Running model rbn_models_for_benchmark/rbn_N_50_ens_5.booleannet
method: stablemotifs2013
method: pysta

In [9]:
result_df.to_excel(output_file_name)

In [10]:
result_df

,stablemotifs2013,pystablemotifs,boolsim,pyboolnet,CABEAN,model
0,2.816108,0.348221,0.202966,0.305287,0.120803,rbn_N_10_ens_0
1,2.397150,0.401876,0.230217,0.288312,0.114602,rbn_N_10_ens_1
2,3.128443,0.360758,0.203329,0.555616,0.117556,rbn_N_10_ens_2
3,5.790938,0.664144,0.206091,0.394895,0.112805,rbn_N_10_ens_3
4,2.626872,0.352216,0.205354,0.700278,0.105271,rbn_N_10_ens_4
5,2.346924,0.208976,0.189192,0.178756,0.087090,rbn_N_10_ens_5
6,2.202775,0.211547,0.191409,0.184920,0.109743,rbn_N_10_ens_6
7,4.059009,0.949475,0.214304,0.501495,0.071754,rbn_N_10_ens_7
8,3.179525,0.353976,0.346806,0.416145,0.136628,rbn_N_10_ens_8
9,4.828619,0.746420,0.207591,0.986826,0.110956,rbn_N_10_ens_9
